# 파라미터 생성 결과 실험용 코드

- 먼저 요놈 돌리쉠

In [1]:
import os
import glob
import json
import numpy as np

from nerf.provider import *
import os
import glob
import json
import tqdm
import numpy as np
from scipy.spatial.transform import Slerp, Rotation


import torch
from nerf.network_tcnn import NeRFNetwork
from nerf.utils import *
from torch.cuda.amp import autocast

def get_opt():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--path', default='./data/photoshapes/shape09096_rank00',type=str)
    parser.add_argument('-O', action='store_true', help="equals --fp16 --cuda_ray --preload")
    parser.add_argument('--test', action='store_true', help="test mode")
    parser.add_argument('--workspace', type=str, default='workspace')
    parser.add_argument('--seed', type=int, default=0)
    ### training options
    parser.add_argument('--iters', type=int, default=15000, help="training iters")
    parser.add_argument('--lr', type=float, default=1e-2, help="initial learning rate")
    parser.add_argument('--ckpt', type=str, default='best')
    parser.add_argument('--num_rays', type=int, default=4096, help="num rays sampled per image for each training step")
    parser.add_argument('--cuda_ray', action='store_true', help="use CUDA raymarching instead of pytorch")
    parser.add_argument('--max_steps', type=int, default=1024, help="max num steps sampled per ray (only valid when using --cuda_ray)")
    parser.add_argument('--num_steps', type=int, default=512, help="num steps sampled per ray (only valid when NOT using --cuda_ray)")
    parser.add_argument('--upsample_steps', type=int, default=0, help="num steps up-sampled per ray (only valid when NOT using --cuda_ray)")
    parser.add_argument('--update_extra_interval', type=int, default=16, help="iter interval to update extra status (only valid when using --cuda_ray)")
    parser.add_argument('--max_ray_batch', type=int, default=4096, help="batch size of rays at inference to avoid OOM (only valid when NOT using --cuda_ray)")
    parser.add_argument('--patch_size', type=int, default=1, help="[experimental] render patches in training, so as to apply LPIPS loss. 1 means disabled, use [64, 32, 16] to enable")
    ### network backbone options
    parser.add_argument('--fp16', action='store_true', help="use amp mixed precision training")
    parser.add_argument('--ff', action='store_true', help="use fully-fused MLP")
    parser.add_argument('--tcnn', action='store_true', help="use TCNN backend")
    ### dataset options
    parser.add_argument('--color_space', type=str, default='srgb', help="Color space, supports (linear, srgb)")
    parser.add_argument('--preload', action='store_true', help="preload all data into GPU, accelerate training but use more GPU memory")
    # (the default value is for the fox dataset)
    parser.add_argument('--bound', type=float, default=2, help="assume the scene is bounded in box[-bound, bound]^3, if > 1, will invoke adaptive ray marching.")
    parser.add_argument('--scale', type=float, default=0.33, help="scale camera location into box[-bound, bound]^3")
    parser.add_argument('--offset', type=float, nargs='*', default=[0, 0, 0], help="offset of camera location")
    parser.add_argument('--dt_gamma', type=float, default=1/128, help="dt_gamma (>=0) for adaptive ray marching. set to 0 to disable, >0 to accelerate rendering (but usually with worse quality)")
    parser.add_argument('--min_near', type=float, default=0.2, help="minimum near distance for camera")
    parser.add_argument('--density_thresh', type=float, default=10, help="threshold for density grid to be occupied")
    parser.add_argument('--bg_radius', type=float, default=-1, help="if positive, use a background model at sphere(bg_radius)")
    ### GUI options
    parser.add_argument('--gui', action='store_true', help="start a GUI")
    parser.add_argument('--W', type=int, default=1920, help="GUI width")
    parser.add_argument('--H', type=int, default=1080, help="GUI height")
    parser.add_argument('--radius', type=float, default=5, help="default GUI camera radius from center")
    parser.add_argument('--fovy', type=float, default=50, help="default GUI camera fovy")
    parser.add_argument('--max_spp', type=int, default=64, help="GUI rendering max sample per pixel")
    ### experimental
    parser.add_argument('--error_map', action='store_true', help="use error map to sample rays")
    parser.add_argument('--clip_text', type=str, default='', help="text input for CLIP guidance")
    parser.add_argument('--rand_pose', type=int, default=-1, help="<0 uses no rand pose, =0 only uses rand pose, >0 sample one rand pose every $ known poses")



    return parser.parse_args(args=[])
opt=get_opt()

opt.fp16 = True
opt.cuda_ray = True
opt.preload = True

opt.test=True

if opt.patch_size > 1:
    opt.error_map = False # do not use error_map if use patch-based training
    # assert opt.patch_size > 16, "patch_size should > 16 to run LPIPS loss."
    assert opt.num_rays % (opt.patch_size ** 2) == 0, "patch_size ** 2 should be dividable by num_rays."

### Network Define
- 이놈도 돌리쉠

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeRFNetwork(
        encoding="hashgrid",
        bound=opt.bound,
        cuda_ray=opt.cuda_ray,
        density_scale=1,
        min_near=opt.min_near,
        density_thresh=opt.density_thresh,
        bg_radius=opt.bg_radius,
    ).to(device)

# ckpt_path를 넣으세요
ckpt_path='./data/photoshape_weight/shape09096_rank01/checkpoints/ngp.pth'
#ckpt_path='hi.pth'
# 저장경로
opt.workspace='workspace/result'

if not os.path.isfile(opt.workspace):
    os.makedirs(opt.workspace,exist_ok=True)

torch.cuda.empty_cache()
ckpt=torch.load(ckpt_path)

model.load_state_dict(ckpt['model'],strict=False)
model.to(device)
model.eval()


opt.path='./data/photoshapes/shape09096_rank01'

# camera parametre defining!
focal=245/(2*np.tan(np.radians(60)/2))
intrinsics=np.array([focal,focal,256//2,256//2])

H,W=256,256

generation_frame=40 # 생성할 뷰 갯수란다.
radius=0.5 # 해당 파라미터를 조정하면 줌업을 할 수 있단다

## Line Pose Generation
- upper hemisphere interpolation 비디오 생성

In [3]:
poses=line_poses(generation_frame,device,radius=radius)
data=get_rays(poses,intrinsics,H,W)

with autocast():
    with torch.no_grad():
        result=model.render(data['rays_o'].float(),data['rays_d'].float())
        

pred=result['image'].reshape(-1,256,256,3)
pred=pred.detach().cpu().numpy()
#pred=linear_to_srgb(pred).detach().cpu().numpy()
#pred=srgb_to_linear(pred).detach().cpu().numpy()
pred=(pred*255).astype(np.uint8)

#cv2.imwrite(os.path.join(opt.workspace,'sample0.png'), cv2.cvtColor(pred[3], cv2.COLOR_RGB2BGR))
imageio.mimwrite(os.path.join(opt.workspace,'sample.mp4'),pred, fps=8, quality=8, macro_block_size=1)


## Random Pose Generation

In [18]:
poses=rand_poses(generation_frame,device,theta_range=[np.pi/3,np.pi/3],radius=radius)
data=get_rays(poses,intrinsics,H,W)
with autocast():
    with torch.no_grad():
        result=model.render(data['rays_o'].float(),data['rays_d'].float())

pred=result['image'].reshape(-1,256,256,3)
pred=srgb_to_linear(pred).detach().cpu().numpy()
pred=(pred*255).astype(np.uint8)

#cv2.imwrite(os.path.join(opt.workspace,'sample1.png'), cv2.cvtColor(pred[3], cv2.COLOR_RGB2BGR))
imageio.mimwrite(os.path.join(opt.workspace,'sample.mp4'),pred, fps=2, quality=8, macro_block_size=1)


## 훈련에 사용한 transform_test.json 파일이 있을경우
- 데이터셋에서 제공하는 camera parameter로 테스트하는 경우란다.

In [ ]:
# 데이터셋 디렉터리 경로
opt.path='./data/photoshapes/shape09096_rank01'

metrics = [PSNRMeter(), LPIPSMeter(device=device)]
test_loader = NeRFDataset(opt, device=device, type='test').dataloader()
trainer = Trainer('ngp', opt, model, device=device, workspace=opt.workspace, fp16=opt.fp16, metrics=metrics, use_checkpoint=opt.ckpt)
trainer.test(test_loader, write_video=True)